# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [6]:
#from tensorflow import keras
#from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Embedding
#from tensorflow.keras.layers import SimpleRNN
#from tensorflow.keras.datasets import imdb
#from tensorflow.keras import initializers
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers
from keras.utils import pad_sequences

In [7]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [8]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [9]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [10]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [11]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [10]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [12]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

/Users/itnas/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [12]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10


2023-03-20 18:56:34.529516: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-20 18:56:34.954084: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - ETA: 0s - loss: 0.6493 - accuracy: 0.6308

2023-03-20 19:01:52.484904: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 349s 444ms/step - loss: 0.6493 - accuracy: 0.6308 - val_loss: 0.5905 - val_accuracy: 0.6904
Epoch 2/10
782/782 [==============================] - 350s 448ms/step - loss: 0.5438 - accuracy: 0.7264 - val_loss: 0.5315 - val_accuracy: 0.7295
Epoch 3/10
782/782 [==============================] - 352s 450ms/step - loss: 0.4834 - accuracy: 0.7695 - val_loss: 0.4982 - val_accuracy: 0.7520
Epoch 4/10
782/782 [==============================] - 349s 447ms/step - loss: 0.4431 - accuracy: 0.7946 - val_loss: 0.4802 - val_accuracy: 0.7626
Epoch 5/10
782/782 [==============================] - 349s 446ms/step - loss: 0.4159 - accuracy: 0.8116 - val_loss: 0.4664 - val_accuracy: 0.7717
Epoch 6/10
782/782 [==============================] - 347s 444ms/step - loss: 0.3964 - accuracy: 0.8227 - val_loss: 0.4535 - val_accuracy: 0.7812
Epoch 7/10
782/782 [==============================] - 348s 445ms/step - loss: 0.3825 - accuracy: 0.8296 - val_loss: 0.4505 - val_accura

In [13]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 33s 42ms/step - loss: 0.4466 - accuracy: 0.7896
Test score: 0.44661474227905273
Test accuracy: 0.7896400690078735


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [13]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [14]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [15]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [18]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10


2023-03-20 20:16:42.150445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - ETA: 0s - loss: 0.5977 - accuracy: 0.6728

2023-03-20 21:21:43.890409: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 4280s 5s/step - loss: 0.5977 - accuracy: 0.6728 - val_loss: 0.5155 - val_accuracy: 0.7422
Epoch 2/10
782/782 [==============================] - 2758s 4s/step - loss: 0.4494 - accuracy: 0.7931 - val_loss: 0.4838 - val_accuracy: 0.7601
Epoch 3/10
782/782 [==============================] - 2554s 3s/step - loss: 0.3851 - accuracy: 0.8290 - val_loss: 0.4368 - val_accuracy: 0.7927
Epoch 4/10
782/782 [==============================] - 2229s 3s/step - loss: 0.3460 - accuracy: 0.8531 - val_loss: 0.4003 - val_accuracy: 0.8165
Epoch 5/10
782/782 [==============================] - 2231s 3s/step - loss: 0.3176 - accuracy: 0.8674 - val_loss: 0.3889 - val_accuracy: 0.8266
Epoch 6/10
782/782 [==============================] - 2232s 3s/step - loss: 0.2971 - accuracy: 0.8775 - val_loss: 0.3796 - val_accuracy: 0.8311
Epoch 7/10
782/782 [==============================] - 2232s 3s/step - loss: 0.2819 - accuracy: 0.8844 - val_loss: 0.3726 - val_accuracy: 0.8341
Epo

In [17]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [18]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [19]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [20]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10


2023-03-21 16:25:05.281470: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - ETA: 0s - loss: 0.6551 - accuracy: 0.6144

2023-03-21 17:02:50.593997: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 2439s 3s/step - loss: 0.6551 - accuracy: 0.6144 - val_loss: 0.5639 - val_accuracy: 0.7193
Epoch 2/10
782/782 [==============================] - 2246s 3s/step - loss: 0.5061 - accuracy: 0.7548 - val_loss: 0.4839 - val_accuracy: 0.7644
Epoch 3/10
782/782 [==============================] - 2231s 3s/step - loss: 0.4430 - accuracy: 0.7944 - val_loss: 0.4576 - val_accuracy: 0.7851
Epoch 4/10
782/782 [==============================] - 2228s 3s/step - loss: 0.4032 - accuracy: 0.8182 - val_loss: 0.4216 - val_accuracy: 0.8028
Epoch 5/10
782/782 [==============================] - 2224s 3s/step - loss: 0.3742 - accuracy: 0.8333 - val_loss: 0.4071 - val_accuracy: 0.8108
Epoch 6/10
782/782 [==============================] - 2200s 3s/step - loss: 0.3521 - accuracy: 0.8484 - val_loss: 0.3926 - val_accuracy: 0.8197
Epoch 7/10
782/782 [==============================] - 2455s 3s/step - loss: 0.3352 - accuracy: 0.8566 - val_loss: 0.3740 - val_accuracy: 0.8331
Epo

In [21]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 2230s 3s/step - loss: 0.2997 - accuracy: 0.8740 - val_loss: 0.3598 - val_accuracy: 0.8406
Epoch 2/10
782/782 [==============================] - 2224s 3s/step - loss: 0.2942 - accuracy: 0.8775 - val_loss: 0.3610 - val_accuracy: 0.8400
Epoch 3/10
782/782 [==============================] - 2223s 3s/step - loss: 0.2903 - accuracy: 0.8790 - val_loss: 0.3578 - val_accuracy: 0.8424
Epoch 4/10
782/782 [==============================] - 2230s 3s/step - loss: 0.2859 - accuracy: 0.8811 - val_loss: 0.3552 - val_accuracy: 0.8436
Epoch 5/10
782/782 [==============================] - 2231s 3s/step - loss: 0.2829 - accuracy: 0.8832 - val_loss: 0.3537 - val_accuracy: 0.8437
Epoch 6/10
782/782 [==============================] - 2232s 3s/step - loss: 0.2802 - accuracy: 0.8828 - val_loss: 0.3620 - val_accuracy: 0.8425
Epoch 7/10
782/782 [==============================] - 2229s 3s/step - loss: 0.2777 - accuracy: 0.8845 - val_loss: 0.3605 - val_accuracy:

In [22]:
model_rnn.predict(x_test)>0.5

2023-03-22 04:58:30.805290: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 173s 221ms/step


array([[False],
       [ True],
       [ True],
       ...,
       [False],
       [False],
       [ True]])

In [23]:
y_test

array([0, 1, 1, ..., 0, 0, 0])

---
### Machine Learning Foundation (C) 2020 IBM Corporation